In [1]:
import tiktoken
enc = tiktoken.get_encoding("gpt2")


In [2]:
enc.encode("A")

[32]

In [ ]:
# ../../video_ve_ozet_havuzu/reproduce_gpt2/tinyshakespeare/tiny_shakespeare.txt

In [3]:
import os
data_filename = os.path.join("", "tiny_shakespeare.txt")
text = open(data_filename, 'r').read()
len(text), text[:100]

(1115394,
 'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou')

In [96]:
letter_list = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z',
               ' ', ',', '.', '!', '?', '-']
len(letter_list)

32

In [97]:
def letter_encoder(letter):
  letter = letter.lower()
  if letter in letter_list:
    return letter_list.index(letter)
  else:
    return -1


In [98]:
def letter_decoder(index):
  return letter_list[index]

In [100]:
letter_decoder(letter_encoder(' '))

' '

In [61]:
def tokenize(text):
  tokens = []
  for letter in text:
    token = letter_encoder(letter)
    if token != -1:
      tokens.append(token)
  # %10 of the tokens are for validation and test set
  val_test_size = len(tokens) // 10
  train_tokens = tokens[:-2*val_test_size]
  val_tokens = tokens[-2*val_test_size:-val_test_size]
  test_tokens = tokens[-val_test_size:]
  # save the tokens as binary files
  with open("train_tokens2.bin", "wb") as f:
    f.write(bytes(train_tokens))
  with open("val_tokens2.bin", "wb") as f:
    f.write(bytes(val_tokens))
  with open("test_tokens2.bin", "wb") as f:
    f.write(bytes(test_tokens))

In [70]:
text_set = set(text.lower().replace('\n', ' ').split(" "))
len(text_set)

23642

In [79]:
from datasets import load_dataset

# load a small dataset for testing from the Hugging Face datasets library
dataset = load_dataset("wikipedia", language="simple", date="20220301")

dataset

Generating train split: 100%|██████████| 205328/205328 [00:00<00:00, 968427.50 examples/s]


DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'text'],
        num_rows: 205328
    })
})

In [80]:
# dataset to text
text = dataset['train']['text']

In [81]:
len(text)

205328

In [84]:
test_str = ""

for i in range(100):
  test_str += text[i].lower().replace('\n', ' ')

len(test_str)

413854

In [88]:
text_set = list(set(test_str.split(" ")))

len(text_set), text_set[:10]


(13513,
 ['',
  'australasia,',
  'phillip,',
  'a\xa0',
  'belgium,',
  'tattoo',
  'personal',
  '0s).',
  'words',
  'gone'])

In [104]:
import numpy as np
# generate a random text contains just 100 different words
# get random words from the text
english_word_set = set()

while len(english_word_set) < 100:
  word = np.random.choice(list(text_set))
  english_word_set.add(word)

english_word_list = list(english_word_set)
english_word_list


['roots',
 'third-largest',
 'jules',
 'grammminology/',
 'africa:',
 'sides',
 'sue',
 'aymará',
 'speed',
 'y',
 'otherwise,',
 'rumi,',
 'functions',
 "(today's",
 'profiles',
 'plateaus',
 'doubted.',
 'abbasids',
 'curve.',
 'savimbi',
 'mays,',
 'foil,',
 'wine.',
 'military',
 'line.',
 'authors,',
 'homo,',
 'dramatically,',
 'lydia',
 '(dame',
 'adds',
 'indivisible,',
 'truly',
 'toots',
 ')',
 'rebellion',
 'table',
 'by.',
 '1804',
 'direction),',
 'artist,',
 'dynasty.',
 'information,',
 'cellular',
 'barcelona)',
 'productive',
 'forth',
 'explorers.',
 'indiana.',
 'authority',
 'heads',
 'ensor',
 'cross-quarter',
 'crossword',
 'casting',
 'few',
 'darkness',
 'explodes',
 'fish-traps',
 'harvest.',
 'inverse',
 'cobs.',
 'trappist',
 'acronyms',
 'osaka,',
 'italian.',
 'kurdish',
 'initiates',
 'toxic.',
 'team',
 'argon,',
 'occitan',
 '1994',
 '():',
 'notations',
 "'new",
 'imprisoned',
 'numbers',
 'industry.',
 'post',
 'toledo)',
 'groups',
 'lambs',
 '“cantor

In [105]:
# create 100 random sentences with 100 words
english_text = ""
for i in range(100):
  english_text += " ".join(np.random.choice(english_word_list, 100)) + " "

len(english_text), english_text[:100]

(87811,
 "information, roots teaching dynasty. inverse industry. dramatically, abbasids (today's aymará third-")

In [106]:
tokenize(english_text)

In [107]:
with open("val_tokens2.bin", "rb") as f:
  val_tokens = f.read()
  val_tokens = [x for x in val_tokens]
  print(len(val_tokens))
  val_text = "".join([letter_decoder(x) for x in val_tokens])
  print(val_text[:100])
  tokens = np.frombuffer(f.read(), dtype=np.uint16)
  print(len(tokens))


8453
s third-largest kurdish  teaching judge britains toxic. explorers. harvest. cellular  grammminology 
0
